In [57]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [62]:
icm = sc.textFile("../icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))

In [66]:
sparseFeatures=CoordinateMatrix(icm)
sparseUsersItem=CoordinateMatrix(trainSet).toBlockMatrix()

In [44]:
indRows=sparseFeatures.toIndexedRowMatrix().rows

In [49]:
def normalize(indexRow):
    length=len(indexRow.vector.values)
    indexRow.vector.values=np.array(list(map(lambda x: x/np.sqrt(length),indexRow.vector.values)))
    return indexRow

indRowsMapped=indRows.map(normalize)
indRowsMapped.take(3)

[IndexedRow(18624, (19716,[3284,5787,16215],[0.57735026919,0.57735026919,0.57735026919])),
 IndexedRow(32196, (19716,[9004,12362,13094,18570,18904],[0.4472135955,0.4472135955,0.4472135955,0.4472135955,0.4472135955])),
 IndexedRow(32432, (19716,[4715,9853,12284],[0.57735026919,0.57735026919,0.57735026919]))]

In [52]:
indRowsMapped.filter(lambda x: x.index==2).take(5)

[IndexedRow(2, (19716,[9004,9857,15230,15634,18904,19606],[0.408248290464,0.408248290464,0.408248290464,0.408248290464,0.408248290464,0.408248290464]))]

In [70]:
sparseFeatures=IndexedRowMatrix(indRowsMapped)
sparseFeatures.rows.take(5)

[IndexedRow(18624, (19716,[3284,5787,16215],[0.57735026919,0.57735026919,0.57735026919])),
 IndexedRow(32196, (19716,[9004,12362,13094,18570,18904],[0.4472135955,0.4472135955,0.4472135955,0.4472135955,0.4472135955])),
 IndexedRow(32432, (19716,[4715,9853,12284],[0.57735026919,0.57735026919,0.57735026919])),
 IndexedRow(7608, (19716,[2483,5787,9416,11179,18904],[0.4472135955,0.4472135955,0.4472135955,0.4472135955,0.4472135955])),
 IndexedRow(9166, (19716,[5787],[1.0]))]

In [72]:
sparseFeaturesBlock=sparseFeatures.toBlockMatrix()
print(sparseFeaturesBlock.numRows())
print(sparseFeaturesBlock.numCols())

37142
19716


In [73]:
print(sparseUsersItem.numRows())
print(sparseUsersItem.numCols())

15375
37143


In [69]:
multiplication=sparseUsersItem.multiply(sparseFeaturesBlock)

IllegalArgumentException: 'requirement failed: The number of columns of A and the number of rows of B must be equal. A.numCols: 37143, B.numRows: 37142. If you think they should be equal, try setting the dimensions of A and B explicitly while initializing them.'

In [ ]:
sparseFeaturesBlock.multiply